In [ ]:
!pip install word2vec




In [51]:
import numpy as np
import scipy
import operator
from keras.preprocessing import sequence, text
from keras.models import Sequential
from keras.layers import Dense, Activation, Embedding, LSTM, SpatialDropout1D, Dropout, Flatten
#from keras.datasets import imdb
import pandas as pd
import numpy as np
#устанавливаем seed
np.random.seed(42)
import pymorphy2
import re
import datetime
import sklearn.feature_extraction.text as fex
morph = pymorphy2.MorphAnalyzer()
from IPython.core.display import display, HTML, Javascript, DisplayObject, display
import lime
import sklearn.pipeline
from lime.lime_text import LimeTextExplainer
import tomita
#from tparser import tomita
from tomita_parser import text_parse
import pandas as pd
def normalize(text):
    return ' '.join([morph.parse(str(w))[0].normal_form for w in text.split()])
def clean_text(text):
    text = str(text)
    text = text.replace("\\", " ").replace(u"╚", " ").replace(u"╩", " ")
    text = text.lower()
    text = re.sub('\-\s\r\n\s{1,}|\-\s\r\n|\r\n', '', text) #deleting newlines and line-breaks
    text = re.sub('[.,:;_%©?*,!@#$%^&()\d]|[+=]|[[]|[]]|[/]|"|\s{2,}|-\'>><`', ' ', text) #deleting symbols  
    text = re.sub('-', ' ', text) #deleting symbols  
    #text = ' '.join(word[:] for word in text.split() if len(word)>3)
    #text = text.encode("utf-8")
    return text 

In [52]:
#df_train = pd.read_csv('dataset1.csv', delimiter=',', encoding='utf-8', error_bad_lines=False)
#df_trainR = pd.read_csv('datasetNormFinal.csv', delimiter=',', encoding='utf-8', error_bad_lines=False)
df_trainR = pd.read_csv('datasetNormFinal_v2.csv', delimiter=',', encoding='utf-8', error_bad_lines=False)



In [53]:
import pickle
textColName = "TEXT"
negatColName =  "DRAWBACKS"
positColName = "BENEFITS"
ratingColName = "RATING"
catColName = "CATEGORY_ID"
brandColName = "BRAND_ID"
catNameColName= "CATEGORY_NAME"
tovarIdColName = "PRODUCT"
fTextColName = 'fullText'
fTextColNameNorm = 'fullTextNorm'
tomitaColName = 'tomitaCol'
normPosColName = "NormPos"
normNegColName = "NormNeg"
normTextColName = 'NormCpom'
prodNameColName = 'NAME'
#df_train[prodNameColName]
with open('noun_adjf_classifiers.pkl', 'rb') as fid:
    f = pickle.load(fid)

/usr/anaconda3/lib/python3.6/site-packages/sklearn/base.py:315: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.18.1 when using version 0.18.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/usr/anaconda3/lib/python3.6/site-packages/sklearn/base.py:315: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.18.1 when using version 0.18.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/usr/anaconda3/lib/python3.6/site-packages/sklearn/base.py:315: UserWarning: Trying to unpickle estimator TfidfTransformer from version 0.18.1 when using version 0.18.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/usr/anaconda3/lib/python3.6/site-packages/sklearn/base.py:315: UserWarning: Trying to unpickle estimator TfidfVectorizer from version 0.18.1 when using version 0.18.2. This might lead to breaking code or inv

In [54]:
df_train=df_trainR.copy()

In [55]:
#category_data = df_train[df_train[catNameColName].str.contains("MARTPHON")==True]


In [56]:
prodIdList = [30025022,30025023,30024754,30026026]

testList = df_train[(df_train[tovarIdColName] == prodIdList[0])]
print(len(testList))

for prId in prodIdList[1:]:
    testList = pd.concat([testList,df_train[(df_train[tovarIdColName] == prId)]])
print(len(testList))
testList.to_csv("newProd2.csv")

44
139


In [57]:
print(len (df_train))
for prId in prodIdList:
    df_train = df_train[(df_train[tovarIdColName] != prId)] 
    #                & (df_train[tovarIdColName] != 30025023 )]
print(len (df_train))

270941
270802


In [58]:
df_test = pd.read_csv('newProd2.csv', delimiter=',', encoding='utf-8', error_bad_lines=False)
colList = pd.unique(df_test[catNameColName])


In [59]:
import gensim
import pandas as pd
wtw = gensim.models.Word2Vec.load('word2vec_mvideo.model')

In [60]:
def avg_feature_vector(words, model, num_features):
        #function to average all words vectors in a given paragraph
        featureVec = np.zeros((num_features,), dtype="float32")
        nwords = 0

        #list containing names of words in the vocabulary
        index2word_set = set(model.wv.index2word)
        #this is moved as input param for performance reasons
        for word in words:
            if word in index2word_set:
                nwords = nwords+1
                featureVec = np.add(featureVec, model[word])

        if(nwords>0):
            featureVec = np.divide(featureVec, nwords)
        return featureVec
    

In [61]:
!pip install spatial

  Could not find a version that satisfies the requirement spatial (from versions: )
No matching distribution found for spatial


In [62]:
#get average vector for sentence 1
from scipy import spatial
#from  spatial import distance
sentence_1 = "телефон хороший"
sentence_1_avg_vector = avg_feature_vector(sentence_1.split(), model=wtw, num_features=100)

#get average vector for sentence 2
sentence_2 = "телефон обязательно покупайте"
sentence_2_avg_vector = avg_feature_vector(sentence_2.split(), model=wtw, num_features=100)

sen1_sen2_similarity =  1 - spatial.distance.cosine(sentence_1_avg_vector,sentence_2_avg_vector)
print(sen1_sen2_similarity)

0.607673750059


In [106]:
def sentSimilar(sent1, sent2):
    sentence_1_avg_vector = avg_feature_vector(normalize(sent1).split(), model=wtw, num_features=100)
    sentence_2_avg_vector = avg_feature_vector(normalize(sent2).split(), model=wtw, num_features=100)
    res= 1 - spatial.distance.cosine(sentence_1_avg_vector,sentence_2_avg_vector)
    return res
#s1 = "ИГРЫ ИДУТ БЕЗ ЛАГОВ ПОЛНОСТЬЮ"
#s2 = "АППАРАТОМ ПОЛНОСТЬЮ ДОВОЛЕН"
#sentSimilar(normalize(s1),normalize(s2))

In [111]:
maxFeatures = 10
maxWft = 4
maxPerSub = 3
coefN = float(0.7)
coefS = float(0.7)
topCount = 15
def word_weight(cv, explainer, words):
    """word - текст (строка)"""
   # words = [str(w) for w in words if w is not None]
    #print("call")
    words = " ".join(words.split(" "))
    #print("splitted")
    exp = explainer.explain_instance(words, cv.predict_proba, num_features=100)
    return {k:v for k,v in exp.as_list()}
        
def printForCat(df_train, df_test,cat, fMod):
    cv = sklearn.pipeline.make_pipeline(fMod[1], fMod[0])
    explainer = LimeTextExplainer(class_names=[0,1])
 
    category_data = df_train[df_train[catNameColName].str.contains(cat)==True].copy()
    category_data[fTextColName] = category_data[textColName].astype('str') + " "+category_data[positColName].astype('str')
    category_data[fTextColName] = category_data[fTextColName]  +" "+category_data[negatColName].astype('str')

    if (normTextColName in df_train.columns):
       # print ("data already normalized")
        category_data[fTextColNameNorm] = category_data[normTextColName].astype('str') + " "+category_data[normNegColName].astype('str')
        category_data[fTextColNameNorm] = category_data[fTextColNameNorm]  +" "+category_data[normPosColName].astype('str')
    else:
        print("start normalize at" + str (datetime.datetime.now()))
        category_data[fTextColNameNorm] = category_data[fTextColName].apply(clean_text).apply(normalize)
        print("finishaed  collect normalize at " + str (datetime.datetime.now().time()))
    category_data_test = df_test[df_test[catNameColName].str.contains(cat)==True].copy()
    category_data_test[fTextColName] = category_data_test[textColName].astype('str') + " "+category_data_test[positColName].astype('str')
    category_data_test[fTextColName] = category_data_test[fTextColName]  +" "+category_data_test[negatColName].astype('str')
    category_data_test[fTextColNameNorm]=category_data_test[fTextColName].apply(clean_text).apply(normalize)
    textAndProdGrS_test = category_data_test.groupby(tovarIdColName )[tovarIdColName, fTextColName].agg(lambda x: " %s " % ', '.join(x)).reset_index()
    textAndProdGrSNorm_test =  category_data_test.groupby(tovarIdColName )[tovarIdColName, fTextColNameNorm].agg(lambda x: " %s " % ', '.join(x)).reset_index()
    textAndProdGrS = category_data.groupby(tovarIdColName )[tovarIdColName, fTextColName].agg(lambda x: " %s " % ', '.join(x)).reset_index()
    textAndProdGrSNorm =  category_data.groupby(tovarIdColName )[tovarIdColName, fTextColNameNorm].agg(lambda x: " %s " % ', '.join(x)).reset_index()
    textAndProdGrSNorm_All = pd.concat([textAndProdGrSNorm,textAndProdGrSNorm_test])
    textAndProdGrS_All = pd.concat([textAndProdGrSNorm,textAndProdGrSNorm_test])
   # print("total category items " + str(len(textAndProdGrSNorm_All)))
    tf_list = np.array(textAndProdGrSNorm_All)
   # print(tf_list.shape)
    id_tlist = textAndProdGrSNorm_test[tovarIdColName].tolist()

    indList = [i for i in range(0,len(tf_list)) if tf_list[i][0] in id_tlist]
    textAndProdGrS_test[tomitaColName] = textAndProdGrS_test[:][fTextColName].apply(text_parse)
    tfidf = fex.TfidfVectorizer()
    #print("start tfidf at" + str (datetime.datetime.now().time()))
    tfIdfMatrix = tfidf.fit_transform(tf_list[:,1])
    #print("end tf Idf at " + str (datetime.datetime.now().time()))
    ftNames = tfidf.get_feature_names()
    
    densM = tfIdfMatrix.todense()
    for ind in indList[:]:
        print("#"*50)
        display(HTML('<h2>Продукт</h2>'))
        f_weigth = word_weight(cv,explainer,tf_list[ind][1])
        display(HTML(df_test[df_test[tovarIdColName] == tf_list[ind][0]][prodNameColName].iloc[0]))
        col = densM[ind].tolist()[0]
        valD = { ftNames[i]:v for i,v in enumerate(col) if v>0  }
        sorted_topWords = sorted(valD.items(), key=operator.itemgetter(1), reverse=True)
        sorted_topWords = [ s for s in sorted_topWords if len(s[0])>3]
        dict_r = { s[0]:s[1] for s in sorted_topWords}
        tomitaD = textAndProdGrS_test[textAndProdGrS_test[tovarIdColName] == textAndProdGrS_All.iloc[ind][tovarIdColName]][tomitaColName]
        sublist = list([x[::,0] for x in tomitaD.values][0])
        set_ret = list()
        tomitaPairs = tomitaD.tolist()[0]
        #print("start word rating rebuild at " + str (datetime.datetime.now().time()))
        setSent = list()
        setSentWord = dict()
        top10List=list()
        for ts in tomitaPairs:
            skip=False


            ret =0
            retBonus = 0
            relationRet = 0
            tsn = normalize(ts[0])
            if tsn in dict_r.keys():
                ret+=dict_r[tsn]*float(coefS)
            if tsn in f_weigth.keys():
                #print("noon ok")
                relationRet+=f_weigth[tsn]*float(coefS)
                if f_weigth[tsn]<0:
                    relationRet+=f_weigth[tsn]*2
                retBonus+=abs(1*f_weigth[tsn]*float(coefS))
            retList = list()
            for tsPr in ts[1].split(" "):
                tsPrn = normalize(tsPr)
                if tsPrn in dict_r.keys():
                    retList.append(dict_r[tsPrn])
                if tsPrn in f_weigth.keys():
                    #print("not noon ok")
                    relationRet+=f_weigth[tsPrn]
                    retBonus+=abs(1*f_weigth[tsPrn])
                    if f_weigth[tsPrn]<0:
                        relationRet+=f_weigth[tsPrn]*2

            #ret+=retBonus
            retList.sort(reverse = True)
            coef = coefN
            for retN in retList:
                ret+=float(retN)*coef
                coef*=coef
            if len(top10List) < topCount:
                top10List.append(ret)
                top10List.sort(reverse=True)
            else:
                if ret <top10List[-1]:
                    continue
                top10List.append(ret)
                top10List.sort(reverse=True)
                del top10List[-1]
            for sent in setSent:
                #print(sent)
                #print(ts[1])
                #print( sentSimilar(sent,ts[1]))
                if sentSimilar(sent,ts[1]) > float(0.8):
                    skip=True
                    break
            if skip:
                #print(sent)
                #print(ts[1])
                #print( sentSimilar(sent,ts[1]))
                continue
            setSent.append(ts[1])
            
            set_ret.append([ts[1],ret,ts[0],relationRet])
        
        #print("end word rating rebuild at " + str (datetime.datetime.now().time()))
        sorted_sent = sorted(set_ret, key=operator.itemgetter(1), reverse=True)
        #print(sorted_sent)
        for ft in sorted_sent:
            if  str(ft[2]) in setSentWord.keys():
                setSentWord[ft[2]]+=ft[1]
            else:
                setSentWord[ft[2]]=ft[1]
        sorted_w_sent = sorted(setSentWord.items(), key=operator.itemgetter(1), reverse=True)

               
        subDict = dict()
        curInd = 0
        htmlStr = """<table border="0"> <tr> <th>Особенности</th>
    <th>Важность</th>
    <th>Отношение</th>
    
    </tr> """
        for feature in sorted_sent:
            if  str(feature[2]) in subDict.keys():
                subDict[feature[2]]+=1
            else:
                subDict[feature[2]]=1
            if (subDict[feature[2]] < maxPerSub):
                featText = list(feature[0].lower())
                featText[0] = featText[0].upper()
                featText = "".join(featText) 
                featRet = int(float(feature[3])*10000) 
                relationText = "Нейтральное" if  featRet == 0 else ("Хорошее" if featRet>0 else "Плохое")
                #+  featText.Substring(1);
                htmlStr +=" <tr><td> "
                htmlStr += "  "+featText+ " </td><td>"
                htmlStr += "  "+str(int(float(feature[1])*100))+ " </td><td>"
                htmlStr += "  "+relationText+ " </td><tr>"

                #print(" особенность: "+str(featText)+ " важность "+ 
                #      str(int(float(feature[1])*100)))
                      #+ "сущ" + str(feature[2]))
                curInd+=1
                
         
            if curInd > maxFeatures:
                break
        display(HTML(htmlStr))
        for we in sorted_w_sent[:maxWft]:
            cnt=0
            display(HTML('<h4>'+we[0]+'</h2>'))

            for ft in sorted_sent:
                if ft[2] == we[0]:
                    cnt+=1
                    featText = list(ft[0].lower())
                    featText[0] = featText[0].upper()
                    featText = "".join(featText)
                    featRet = int(float(ft[3])*10000) 
                    relationText = "Нейтрально: " if  featRet == 0 else ("Хорошо: " if featRet>0 else "Плохо: ")
                    featText= relationText + featText
                    print(featText)
                    if cnt > maxPerSub:
                        break
                      

In [112]:
import gensim
import pandas as pd
dfTest = df_train[normTextColName]

In [113]:


cat = colList[0][1:]
#print(colList)
for cat in colList[:1]:
    fMod = f['phone']
    #print(fMod[1])
    display(HTML('<h1>'+str(cat) +'</h1>'))
    printForCat(df_train, df_test,cat[1:],fMod)

##################################################


/usr/anaconda3/lib/python3.6/re.py:212: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)


Особенности,Важность,Отношение
Был телефон с емкой батареей,38,Нейтральное
Отличное качество для такой стоимости телефона мощная батарея,38,Хорошее
Хочет приобрести недорогой аппарат с хорошей камерой и мощным зарядом батареи,38,Хорошее
Батарея телефона слабая,37,Плохое
Батареями как например этот,33,Нейтральное
Еще уже третий день в настройках не могу найти будильник неплохая батарея камера,33,Хорошее
Батареи тоже хватает на долго,33,Нейтральное
Батарею держит дольше,32,Нейтральное
В остальном очень хороший телефон,31,Хорошее
Раньше только мечтать приходилось о батарее,31,Нейтральное


Хорошо: Отличное качество для такой стоимости телефона мощная батарея
Плохо: Батарея телефона слабая
Хорошо: Батарея держит часов 55-60 с включенным интернетом хотя я периодически играю
Хорошо: Хорошая батарея


Хорошо: В остальном очень хороший телефон
Нейтрально: Телефон за эти деньги просто
Нейтрально: Телефон который у меня был
Нейтрально: Ранее был телефон


...

Особенности,Важность,Отношение
Камера очень очень слабая,35,Хорошее
Камера отличная 13 мп и 5 мп.телефоном доволен,32,Хорошее
Можно скачать камеру,31,Хорошее
Камера плохая и видео плохого качества,28,Плохое
Камеры не советую,28,Хорошее
Камеры нормальные,28,Хорошее
Фото или видео для себя тогда лучше,28,Хорошее
Фото-видео камера средняя если привыкнуть к автофокусу то оновная камера в дневное время делает хорошие снимки и видео,26,Хорошее
Телефон хороший,25,Хорошее
Темное время на близком растоянии можно получить не плохое фото,24,Плохое


Хорошо: Камера очень очень слабая
Хорошо: Камера отличная 13 мп и 5 мп.телефоном доволен
Хорошо: Лишних программы и отстойная камера


Хорошо: Фото или видео для себя тогда лучше
Плохо: Темное время на близком растоянии можно получить не плохое фото
Плохо: Фото расплывчитые не чёткие


Хорошо: Телефон хороший
Плохо: Пожалела что купила этот телефон
Плохо: Телефон прекрасный


Нейтрально: Видео вообще молчу
Хорошо: Любят смотреть и делать прекрасные фотографии и видео
Хорошо: Видео достойное


In [ ]:
#listN = dfTest.tolist()
import gensim
import pandas as pd
entries = [[str(w) for w in e.split()] for e in df.NormCpom]
wtw = gensim.models.Word2Vec(entries, size=100, window=5, min_count=5, workers=4)
wtw.save('word2vec_mvideo.model')
#print(wtw.wv.most_similar(positive=['хороший'], negative=['плохой']))
#print(wtw.similarity('отличный', 'ужасный'))

In [ ]:
strOut = ("""
<html>
  <head> </head>
  <body>
    <div id="sample-1">Клик по ссылке 1</div>
    <div id="sample-2" style='display:none;'>Клик по ссылке 2</div>
    <div id="sample-3" style='display:none;'>Клик по ссылке 3</div>
    
    
    <ul>
      <li onclick="view('sample-1')">17678687</li>
      <li onclick="view('sample-2')">29878998789</li>
      <li onclick="view('sample-3')">8979777</li>
    </ul>
    <script>
 
      function view(idDiv){
        var divs = document.getElementsByTagName('div');
        for(var i = 0; i < divs.length; i++){
          if(divs[i].id == idDiv){
            divs[i].style.display = 'block';
            continue;
          }
          divs[i].style.display = 'none';
        }
      }
 
 
    </script>
 
  </body>
</html>

""")

display(HTML(strOut))

In [ ]:
from IPython.display import display, HTML
js = ""
#<script>alert('Hello World!');</script>"
display(HTML(js))

In [ ]:
import pickle

In [ ]:
f = dict(f)